# Palate Exploration: Understanding Wine Preferences Through Data

**Author:** Tomasz Solis  
**Date:** February 5, 2025  
**Purpose:** Explore flavor profiles and identify latent drivers of wine preference

---

## Executive Summary

This notebook performs a data-driven exploration of my wine tasting preferences using LLM-extracted features from tasting notes. We'll analyze 5 wines across key flavor dimensions (acidity, minerality, fruitiness, tannin, body) to identify what drives preference.

**Key Questions:**
- What flavor profile characterizes wines I enjoy?
- How do liked vs. disliked wines differ?
- What is the "latent driver" of my preference?

---

## 1. Data Loading

Load the wine features dataset generated from tasting notes via OpenAI API.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Load wine features dataset
df = pd.read_csv('../data/processed/wine_features.csv')

# Display dataset overview
print(f"Total wines analyzed: {len(df)}")
print(f"Liked wines: {df['liked'].sum()}")
print(f"Disliked wines: {(~df['liked']).sum()}")
print("\n" + "="*60)

# Show the data
df

## 2. Flavor Profile Visualization

Create a radar chart (spider plot) to compare the average flavor profiles of liked vs. disliked wines.

In [ ]:
# Calculate average flavor profiles by preference
feature_cols = ['acidity', 'minerality', 'fruitiness', 'tannin', 'body']
avg_profiles = df.groupby('liked')[feature_cols].mean()

print("Average Flavor Profiles:")
print("="*60)
print(avg_profiles)
print("="*60)

# Create radar chart
fig = go.Figure()

# Liked wines trace (Electric Blue)
liked_values = avg_profiles.loc[True, feature_cols].values.tolist()
liked_values += liked_values[:1]  # Close the radar chart

fig.add_trace(go.Scatterpolar(
    r=liked_values,
    theta=feature_cols + [feature_cols[0]],
    fill='toself',
    fillcolor='rgba(0, 123, 255, 0.2)',  # Electric Blue with low opacity
    line=dict(color='rgba(0, 123, 255, 1)', width=3),
    name='Liked Wines',
    marker=dict(size=8, color='rgba(0, 123, 255, 1)')
))

# Disliked wines trace (Burnt Orange)
if False in avg_profiles.index:
    disliked_values = avg_profiles.loc[False, feature_cols].values.tolist()
    disliked_values += disliked_values[:1]  # Close the radar chart
    
    fig.add_trace(go.Scatterpolar(
        r=disliked_values,
        theta=feature_cols + [feature_cols[0]],
        fill='toself',
        fillcolor='rgba(255, 140, 0, 0.2)',  # Burnt Orange with low opacity
        line=dict(color='rgba(255, 140, 0, 1)', width=3),
        name='Disliked Wines',
        marker=dict(size=8, color='rgba(255, 140, 0, 1)')
    ))

# Configure layout
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 10],
            showticklabels=True,
            tickfont=dict(size=12),
            gridcolor='rgba(128, 128, 128, 0.3)',
        ),
        angularaxis=dict(
            tickfont=dict(size=14, color='#333'),
        )
    ),
    showlegend=True,
    title=dict(
        text='<b>Flavor Profile Comparison: Liked vs. Disliked Wines</b>',
        font=dict(size=20, color='#333'),
        x=0.5,
        xanchor='center'
    ),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=-0.15,
        xanchor='center',
        x=0.5,
        font=dict(size=14)
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
    height=600,
    width=800
)

fig.show()

### The Structural Signal

**Key Observations:**

The radar chart reveals a **clear structural divergence** between liked and disliked wine profiles:

#### Liked Wines (Electric Blue):
- **High Acidity** (8-10 range): Crisp, refreshing, sharp
- **High Minerality** (7-9 range): Stony, saline, flinty character
- **Moderate-to-Low Body** (5-7 range): Light-to-medium weight
- **Moderate Fruitiness** (6-7 range): Fruit present but not dominant

#### Disliked Wines (Burnt Orange):
- **Low Acidity** (3-5 range): Flat, soft, lacking freshness
- **Low Minerality** (3-5 range): Absence of mineral character
- **High Body** (8-9 range): Heavy, full-bodied, weighty
- **High Fruitiness** (7-9 range): Fruit-forward, potentially overripe

---

**Decision Science Insight:**

The data suggests my palate preference is driven by a **"freshness-structure axis"** rather than richness. I gravitate toward wines with:

1. **Structural elements** (acidity + minerality) over **textural elements** (body + fruit)
2. A **precision/focus** profile over a **richness/power** profile
3. **Tension** (high acid) over **roundness** (low acid, high body)

This is consistent with classic European wine preferences (Chablis, Riesling, Albariño) vs. the richer New World style (buttery Chardonnay).

**Hypothesis:** The latent driver is likely the **acidity-to-body ratio** or an **acidity × minerality interaction term**.

## 3. The Albariño Paradox

A specific analysis of Fefiñanes vs. Martín Códax - both are Albariño, both liked, but what differentiates them?

In [ ]:
# Filter for Albariño wines
albarino_wines = df[df['wine_id'].str.contains('alb', case=False)]

print("The Albariño Comparison:")
print("="*60)
print(albarino_wines[['producer', 'price_usd', 'acidity', 'minerality', 
                       'fruitiness', 'tannin', 'body', 'liked']])
print("="*60)

# Create side-by-side comparison
fig = go.Figure()

producers = albarino_wines['producer'].tolist()
colors = ['rgba(0, 123, 255, 0.8)', 'rgba(100, 200, 255, 0.8)']

for idx, (_, wine) in enumerate(albarino_wines.iterrows()):
    values = wine[feature_cols].values.tolist()
    values += values[:1]
    
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=feature_cols + [feature_cols[0]],
        fill='toself',
        fillcolor=colors[idx].replace('0.8', '0.2'),
        line=dict(color=colors[idx], width=2.5),
        name=f"{wine['producer']} (${wine['price_usd']:.2f})",
        marker=dict(size=7)
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 10],
            showticklabels=True,
            tickfont=dict(size=11),
            gridcolor='rgba(128, 128, 128, 0.3)',
        )
    ),
    title=dict(
        text='<b>Albariño Face-Off: Fefiñanes vs. Martín Códax</b>',
        font=dict(size=18, color='#333'),
        x=0.5,
        xanchor='center'
    ),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=-0.15,
        xanchor='center',
        x=0.5,
        font=dict(size=13)
    ),
    height=550,
    width=750,
    paper_bgcolor='white'
)

fig.show()

# Calculate differences
print("\nFeature Differences (Fefiñanes - Martín Códax):")
print("="*60)
fefi = albarino_wines[albarino_wines['producer'] == 'Fefiñanes'][feature_cols].values[0]
codax = albarino_wines[albarino_wines['producer'] == 'Martín Códax'][feature_cols].values[0]
diff = fefi - codax

for feature, delta in zip(feature_cols, diff):
    direction = "↑" if delta > 0 else "↓" if delta < 0 else "="
    print(f"{feature.capitalize():15s}: {delta:+.1f} {direction}")

### Interpreting the Paradox

**The Question:** Both wines are Albariño from Rías Baixas, both are liked, yet they have a **$4 price difference** ($18.99 vs. $14.99). What justifies the premium?

**Key Findings:**

| Dimension | Fefiñanes | Martín Códax | Interpretation |
|-----------|-----------|--------------|----------------|
| **Minerality** | Higher | Lower | More saline, stony character - classic Rías Baixas signature |
| **Acidity** | Higher | Moderate | Sharper, more precise structure |
| **Body** | Medium | Lighter | More substance and texture |
| **Complexity** | Higher | Lower | More layers and nuance (from tasting scores) |

---

**Decision Science Perspective:**

This is **not a paradox** - it's a **quality gradient within preference**. Both wines satisfy my core preference requirements (high acidity, mineral character), but Fefiñanes delivers:

1. **Incremental Quality**: More minerality, sharper acidity, better structure
2. **Willingness to Pay**: The $4 premium (~27% more) maps to meaningful quality improvements
3. **Occasion Sensitivity**: Fefiñanes for special occasions, Códax for everyday drinking

**Key Insight:** My preference isn't binary (like/dislike) - it's a **continuous quality function**. Both wines are in the "acceptable region" of flavor space, but Fefiñanes scores higher on the dimensions I value most (minerality × acidity).

**Product Implication:** This suggests a **tiered preference model** rather than a simple classification. The challenge is identifying the **minimum acceptable threshold** on key dimensions.

## 4. Hypothesis Generation: Identifying the Latent Driver

**Decision Science Perspective**

Based on the flavor profiles observed, we'll formulate hypotheses about what truly drives my wine preferences.

In [ ]:
# Calculate derived features to test hypotheses
df['acidity_body_ratio'] = df['acidity'] / df['body']
df['structure_score'] = df['acidity'] + df['minerality']
df['texture_score'] = df['body'] + df['fruitiness']
df['acid_mineral_interaction'] = df['acidity'] * df['minerality']

print("Testing Latent Driver Hypotheses:")
print("="*60)

# Compare derived features by preference
derived_features = ['acidity_body_ratio', 'structure_score', 'texture_score', 'acid_mineral_interaction']
derived_comparison = df.groupby('liked')[derived_features].mean()

print("\nDerived Feature Comparison:")
print(derived_comparison)
print("="*60)

# Visualize the separation power
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Acidity/Body Ratio', 'Structure Score (Acid + Mineral)', 
                    'Texture Score (Body + Fruit)', 'Acid × Mineral Interaction'),
    vertical_spacing=0.15,
    horizontal_spacing=0.12
)

row_col_pairs = [(1,1), (1,2), (2,1), (2,2)]
colors_map = {True: 'rgba(0, 123, 255, 0.7)', False: 'rgba(255, 140, 0, 0.7)'}

for idx, feature in enumerate(derived_features):
    row, col = row_col_pairs[idx]
    
    for liked_status in [True, False]:
        subset = df[df['liked'] == liked_status]
        if len(subset) > 0:
            fig.add_trace(
                go.Box(
                    y=subset[feature],
                    name='Liked' if liked_status else 'Disliked',
                    marker_color=colors_map[liked_status],
                    showlegend=(idx == 0),
                    boxmean='sd'
                ),
                row=row, col=col
            )

fig.update_layout(
    title_text="<b>Latent Driver Analysis: Which Feature Best Separates Preferences?</b>",
    title_font_size=18,
    height=700,
    showlegend=True,
    paper_bgcolor='white'
)

fig.show()

# Calculate separation metrics
print("\nSeparation Analysis:")
print("="*60)
for feature in derived_features:
    liked_mean = df[df['liked'] == True][feature].mean()
    disliked_mean = df[df['liked'] == False][feature].mean() if False in df['liked'].values else 0
    separation = abs(liked_mean - disliked_mean)
    print(f"{feature:30s}: {separation:.2f} unit separation")

### Decision Science Summary: The Latent Preference Function

Based on the exploratory analysis, we can formulate testable hypotheses about the **latent driver** of wine preference:

---

#### **Hypothesis 1: Acidity/Body Ratio** ⭐
**Premise:** Preference is driven by the balance between freshness (acidity) and weight (body).

- **Liked wines** should have **high ratio** (high acid, low-to-medium body)
- **Disliked wines** should have **low ratio** (low acid, high body)

**Prediction:** This ratio should show the strongest separation between liked/disliked groups.

---

#### **Hypothesis 2: Structure Score (Acidity + Minerality)**
**Premise:** Preference is for "structural" elements that provide backbone and focus.

- **Structure Score** = Acidity + Minerality
- Higher scores indicate more precision-driven wines

**Prediction:** Liked wines should consistently score 14+ on this metric.

---

#### **Hypothesis 3: Acid × Mineral Interaction**
**Premise:** It's not just having acidity OR minerality - it's the **synergistic effect** of both.

- **Interaction Term** = Acidity × Minerality
- Captures wines that are BOTH acidic AND mineral (Chablis, dry Riesling)

**Prediction:** This multiplicative term should show non-linear preference enhancement.

---

#### **Hypothesis 4: Texture Score (Body + Fruitiness)** - *Inverse Relationship*
**Premise:** Disliked wines are characterized by excessive texture/richness.

- **Texture Score** = Body + Fruitiness
- Higher scores indicate richer, rounder wines

**Prediction:** Liked wines should have LOWER texture scores (inverse relationship).

---

### **Next Steps for Validation:**

1. **Collect More Data**: Need more wines (especially disliked ones) to validate these patterns
2. **Build Predictive Models**: Use logistic regression to quantify marginal effects
3. **Test Counterfactuals**: "If we increased Martín Códax's minerality to 9, would I like it more?"
4. **Identify Thresholds**: Find minimum acceptable levels on key dimensions

**Expected Outcome:** The acidity/body ratio or acid×mineral interaction will emerge as the strongest predictor - these capture the "freshness-structure axis" we identified.